In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
import ttkbootstrap as tb
from ttkbootstrap.constants import *
from tkinter import messagebox

# Load Dataset
books = pd.read_csv("Books.csv", sep=";", encoding="latin-1", on_bad_lines="skip", low_memory=False)
ratings = pd.read_csv("Ratings.csv", sep=";", encoding="latin-1", on_bad_lines="skip", low_memory=False)

books.columns = books.columns.str.strip()
ratings.columns = ratings.columns.str.strip()
books['ISBN'] = books['ISBN'].astype(str).str.strip()
ratings['ISBN'] = ratings['ISBN'].astype(str).str.strip()
books = books.drop_duplicates(subset=['ISBN'])

# Filter Dataset
active_users = ratings['User-ID'].value_counts()
active_users = active_users[active_users >= 50].index
ratings = ratings[ratings['User-ID'].isin(active_users)]

popular_books = ratings['ISBN'].value_counts()
popular_books = popular_books[popular_books >= 100].index
ratings = ratings[ratings['ISBN'].isin(popular_books)]

# User-Item Matrix
user_item_matrix = ratings.pivot_table(
    index='User-ID',
    columns='ISBN',
    values='Rating'
).fillna(0)
print("Available User-IDs:", user_item_matrix.index[:20])  # show first 20 valid users

# Matrix Factorization (SVD)
svd = TruncatedSVD(n_components=20, random_state=42)
matrix = svd.fit_transform(user_item_matrix)
approx_ratings = np.dot(matrix, svd.components_)
pred_ratings = pd.DataFrame(
    approx_ratings,
    index=user_item_matrix.index,
    columns=user_item_matrix.columns
)

# Recommendation Function
def recommend_books(user_id, num_recommendations=5):
    if user_id not in pred_ratings.index:
        return None
    user_ratings = pred_ratings.loc[user_id].sort_values(ascending=False)
    rated_books = ratings[ratings['User-ID'] == user_id]['ISBN'].tolist()
    recommendations = user_ratings.drop(rated_books, errors="ignore").head(num_recommendations)
    recs = pd.DataFrame(recommendations).reset_index()
    recs.columns = ['ISBN', 'Predicted Rating']
    recs = recs.merge(books[['ISBN', 'Title']], on='ISBN', how='left')
    return recs

# GUI Part 
def get_recommendations():
    try:
        user_id = int(user_entry.get())
        num_rec = int(num_entry.get())
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter valid numeric values.")
        return

    recs = recommend_books(user_id, num_rec)
    for row in tree.get_children():
        tree.delete(row)

    if recs is None or recs.empty:
        messagebox.showinfo("No Recommendations", "User not found or no recommendations available.")
    else:
        for _, row in recs.iterrows():
            tree.insert("", "end", values=(row['ISBN'], row['Title'], round(row['Predicted Rating'], 2)))


# Create main window
root = tb.Window(themename="litera")  
root.title("📚 Book Recommendation System")
root.geometry("850x500")

# Title
title_lbl = tb.Label(root, text="📖 Personalized Book Recommender",
                     font=("Helvetica", 18, "bold"))
title_lbl.pack(pady=15)

# Input Frame
frame = tb.Frame(root, padding=10)
frame.pack(fill="x", pady=5)

tb.Label(frame, text="Enter User-ID:", font=("Helvetica", 11)).pack(side="left", padx=5)
user_entry = tb.Entry(frame, width=12, font=("Helvetica", 11))
user_entry.pack(side="left", padx=5)

# Prefill valid sample user
sample_user = user_item_matrix.index[0]
user_entry.insert(0, str(sample_user))

tb.Label(frame, text="Number of Recommendations:", font=("Helvetica", 11)).pack(side="left", padx=5)
num_entry = tb.Entry(frame, width=6, font=("Helvetica", 11))
num_entry.pack(side="left", padx=5)
num_entry.insert(0, "5")

tb.Button(frame, text="🔍 Get Recommendations", bootstyle=SUCCESS, command=get_recommendations).pack(side="left", padx=10)

# Results Table (Treeview styled)
cols = ("ISBN", "Title", "Predicted Rating")
tree = tb.Treeview(root, columns=cols, show="headings", height=15, bootstyle=INFO)
for col in cols:
    tree.heading(col, text=col)
    tree.column(col, width=250 if col == "Title" else 150, anchor="center")
tree.pack(fill="both", expand=True, padx=15, pady=15)

root.mainloop()


Available User-IDs: Index([ 243,  254,  507,  626,  638,  741,  882,  929, 1025, 1211, 1424, 1435,
       1674, 1733, 1848, 1903, 2030, 2033, 2110, 2179],
      dtype='int64', name='User-ID')
